# Preparing, Cleaning, and Merging the Datasets

## Download data
### Warning! The below cells will download 321 MB to your machine.

The data is sourced from [this page](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.2024.html) on the Census bureau's website.

If you examine the data yourself, make sure you use the data under "Data and Documents", not under "Replicate Weight Data and Documents".

2014 Data - Uses Redesigned ASCII Data File & Data Dictionary
2015 thru 2018 - Uses ASCII Data File & Data Dictionary
2019 thru 2024 - Uses ASCII Data File & Household, Family, and Person Text Layouts (codex/data dictionary text file in previous years)

In [ ]:
import requests, os

# Generate directories
os.makedirs('input/', exist_ok=True)
os.makedirs('output/', exist_ok=True)
os.makedirs('output/codex/', exist_ok=True)
os.makedirs('output/fwf/', exist_ok=True)
os.makedirs('output/merged/', exist_ok=True)

def get_file(url, file_path):
    print(f"Fetching from {url}")
    r = requests.get(url)
    
    data = r.content
    
    # Skip file if it already exists with same data length
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            if len(data) == len(f.read()):
                print("Already exists! Skipping...")
                return

    with open(file_path, 'wb+') as f:
        print(f"Writing {len(data)} bytes to {file_path}")
        f.write(data)
    print(f"Saved to {file_path}")

# Download 2014 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2014/march/asec2014_pubuse_3x8_rerun_v2.zip', 'input/2014_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2014/march/asec2014R_pubuse.dd.txt', 'input/2014_codex.txt')

# Download 2015 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2015/march/asec2015_pubuse.zip', 'input/2015_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2015/march/asec2015early_pubuse.dd.txt', 'input/2015_codex.txt')

# Download 2016 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2015/march/asec2015_pubuse.zip', 'input/2016_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2016/march/Asec2016_Data_Dict_Full.txt', 'input/2016_codex.txt')

# Download 2017 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2017/march/asec2017_pubuse.zip', 'input/2017_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2017/march/08ASEC2017_Data_Dict_Full.txt', 'input/2017_codex.txt')

# Download 2018 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2018/march/asec2018_pubuse.zip', 'input/2018_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2018/march/08ASEC2018_Data_Dict_Full.txt', 'input/2018_codex.txt')

# Download 2019 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2019/march/asec2019_pubuse.zip', 'input/2019_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2019/march/hhldfmt.txt', 'output/codex/2019_hhldfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2019/march/famlfmt.txt', 'output/codex/2019_famlfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2019/march/persfmt.txt', 'output/codex/2019_persfmt.txt')

# Download 2020 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2020/march/asec2020_pubuse.zip', 'input/2020_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2020/march/hhldfmt.txt', 'output/codex/2020_hhldfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2020/march/famlfmt.txt', 'output/codex/2020_famlfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2020/march/persfmt.txt', 'output/codex/2020_persfmt.txt')

# Download 2021 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2021/march/asec2021_pubuse.zip', 'input/2021_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2021/march/hhldfmt.txt', 'output/codex/2021_hhldfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2021/march/famlfmt.txt', 'output/codex/2021_famlfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2021/march/persfmt.txt', 'output/codex/2021_persfmt.txt')

# Download 2022 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2022/march/asec2022_pubuse.zip', 'input/2022_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2022/march/hhldfmt.txt', 'output/codex/2022_hhldfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2022/march/famlfmt.txt', 'output/codex/2022_famlfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2022/march/persfmt.txt', 'output/codex/2022_persfmt.txt')

# Download 2023 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2023/march/asec2023_pubuse.zip', 'input/2023_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2023/march/hhldfmt.txt', 'output/codex/2023_hhldfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2023/march/famlfmt.txt', 'output/codex/2023_famlfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2023/march/persfmt.txt', 'output/codex/2023_persfmt.txt')

# Download 2024 FWF DAT and codex files
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2024/march/asec2024_pubuse.zip', 'input/2024_data.zip')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2024/march/hhldfmt.txt', 'output/codex/2024_hhldfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2024/march/famlfmt.txt', 'output/codex/2024_famlfmt.txt')
get_file('https://www2.census.gov/programs-surveys/cps/datasets/2024/march/persfmt.txt', 'output/codex/2024_persfmt.txt')

## Unzip data files
### Warning! On top of the 321 MB downloaded in the last step, this cell will extract an additional 3.3 GB from the zip archives.

In [ ]:
import zipfile

for year in range(2014, 2025):
    try:
        with zipfile.ZipFile(f'input/{year}_data.zip', 'r') as zip_file:
            data_file = zip_file.filelist[0]
            data_file.filename = f'{year}_data.DAT'
            print(f"Extracting and decompressing {data_file.file_size} bytes of data from {year}_data.zip")
            zip_file.extract(data_file, "output/fwf")
    except FileNotFoundError:
        print(f"Error: Zip file 'input/{year}_data.zip' not found.")
    except zipfile.BadZipFile:
         print(f"Error: 'input/{year}_data.zip' is not a valid zip file.")
    except Exception as e:
        print(f"An error occurred: {e}")

## Converting fixed-width files to Pandas Dataframe

In [54]:
import pandas as pd
import numpy as np
from io import StringIO
import re

### Set up functions to convert and split each .dat file (for the years 2014, 2015, 2016, 2018) into 2 separate dataframes (one for each record type:
 ## Family, Person)
 ## *Household record will not be used in this project

### Extract these columns only

FAMILY_NUMERIC_COLS = ['FFPOS', 'FH-SEQ', 'FPERSONS', 'FPOVCUT', 'FAMLIS', 'POVLL', 'FTOTVAL', 'FEARNVAL']
PERSON_NUMERIC_COLS = ['PERIDNUM', 'PF-SEQ', 'PH-SEQ', 'A-AGE', 'PEAFEVER', 'A-HGA', 'A-MJOCC', 'PEARNVAL', 'WSAL-VAL', 'DIV-VAL', 'RTM-VAL']

### Splits data dictionary into 3 separate data dictionaries by record type
 ## ---------------------------------------------------------------------------------------------------------------------------------------------
def split_dictionary_by_record(input_path, year):
    with open(input_path, 'r') as f:
        lines = f.readlines()

    record_sections = {'HOUSEHOLD RECORD': [], 'FAMILY RECORD': [], 'PERSON RECORD': []}
    current_section = None

    for line in lines:
        line = line.strip()
        if line in record_sections:
            current_section = line
        elif current_section:
            record_sections[current_section].append(line + '\n')

    with open(f'output/codex/{year}_hhldfmt.txt', 'w+') as f:
        f.writelines(record_sections['HOUSEHOLD RECORD'])
    with open(f'output/codex/{year}_famlfmt.txt', 'w+') as f:
        f.writelines(record_sections['FAMILY RECORD'])
    with open(f'output/codex/{year}_persfmt.txt', 'w+') as f:
        f.writelines(record_sections['PERSON RECORD'])

### Helper function: matches appropriate record location from the data dictionary based on each column to be extracted
 ## ---------------------------------------------------------------------------------------------------------------------------------------------
def extract_layout_from_dict(file_path, features, pattern):
    layout = []
    feature_set = set(features)
    with open(file_path, 'r') as f:
        for line in f:
            match = re.match(pattern, line)
            if match:
                name, size, start = match.groups()
                if name in feature_set:
                    size = int(size)
                    start = int(start) - 1  # Adjusted for 0-based indexing
                    end = start + size
                    layout.append((name, start, end))
    return layout

### Helper function: decodes .fwf format to dataframe using the extracted layout
 ## ---------------------------------------------------------------------------------------------------------------------------------------------
def parse_record_lines(lines, layout, numeric_cols):
    colspecs = [(start, end) for (_, start, end) in layout]
    names = [name for (name, _, _) in layout]
    df = pd.read_fwf(StringIO(''.join(lines)), colspecs=colspecs, names=names)
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

### Takes .fwf file and data dictionaries (split by record) as input, and outputs respective dataframes
 ## ---------------------------------------------------------------------------------------------------------------------------------------------
def parse_asec_fixed_width(filepath, family_dict_path, person_dict_path, family_cols, person_cols, pattern):
    family_layout = extract_layout_from_dict(family_dict_path, family_cols, pattern)
    person_layout = extract_layout_from_dict(person_dict_path, person_cols, pattern)

    with open(filepath, 'r') as f:
        lines = [line for line in f if not line.startswith('*')]

    # Split by record type
    family_lines = [line for line in lines if line.startswith('2')]
    person_lines = [line for line in lines if line.startswith('3')]

    fam_df = parse_record_lines(family_lines, family_layout, family_cols)
    person_df = parse_record_lines(person_lines, person_layout, person_cols)

    return fam_df, person_df

In [ ]:
# Convert and load dataframes from fixed-width files
# ---------------------------------------------------------------------------------------------------------------------------------------------

dataframes = {}

family_cols_2014 = ['FFPOS', 'FH-SEQ', 'FPERSONS',
                    'FPOVCUT', 'FAMLIS', 'POVLL', 'FTOTVAL', 'FEARNVAL']
person_cols_2014 = ['PERIDNUM', 'PF-SEQ', 'PH-SEQ', 'A-AGE', 'PEAFEVER',
                    'A-HGA', 'A-MJOCC', 'PEARNVAL', 'WSAL-VAL', 'DIV-VAL', 'RTM-VAL']

# 2014 data needs a different column mapping so it is handled separately
print("Processing 2014 data to DataFrame...")
split_dictionary_by_record(f'input/2014_codex.txt', '2014')
dataframes['2014'] = {}
dataframes['2014']['fam'], dataframes['2014']['per'] = parse_asec_fixed_width(
    'output/fwf/2014_data.DAT', 'output/codex/2014_famlfmt.txt', f'output/codex/2014_persfmt.txt', family_cols_2014, person_cols_2014, r"D\s+([\w-]+)\s+(\d+)\s+(\d+)")

# Replace hyphens in 2014 columns with underscores
dataframes['2014']['fam'].rename(columns={'FH-SEQ': 'FH_SEQ'}, inplace=True)
dataframes['2014']['per'].rename(columns={'PF-SEQ': 'PF_SEQ', 'A-AGE': 'A_AGE', 'A-HGA': 'A_HGA', 'A-MJOCC': 'A_MJOCC', 'PH-SEQ': 'PH_SEQ', 'DIV-VAL': 'DIV_VAL', 'RTM-VAL': 'RTM_VAL', 'WSAL-VAL': 'WSAL_VAL'}, inplace=True)
dataframes['2014']['per']['CAP_VAL'] = np.nan
dataframes['2014']['per']['YEAR'] = 2014
dataframes['2014']['fam']['YEAR'] = 2014


# after 2014 they use underscores instead of hyphens
family_cols = ['FFPOS', 'FH_SEQ', 'FPERSONS',
               'FPOVCUT', 'FAMLIS', 'POVLL', 'FTOTVAL', 'FEARNVAL']
person_cols = ['PERIDNUM', 'PF_SEQ', 'PH_SEQ', 'A_AGE', 'PEAFEVER',
               'A_HGA', 'A_MJOCC', 'PEARNVAL', 'WSAL_VAL', 'DIV_VAL', 'RTM_VAL']

# can automate for 2015-2018 as they are all formatted the same
for year in ['2015', '2016', '2017', '2018']:
    print(f"Processing {year} data to DataFrame...")
    split_dictionary_by_record(f'input/{year}_codex.txt', year)
    dataframes[year] = {}
    dataframes[year]['fam'], dataframes[year]['per'] = parse_asec_fixed_width(
        f'output/fwf/{year}_data.DAT', f'output/codex/{year}_famlfmt.txt', f'output/codex/{year}_persfmt.txt', family_cols, person_cols, r"D\s+([\w-]+)\s+(\d+)\s+(\d+)")
    
    dataframes[year]['per']['CAP_VAL'] = np.nan
    dataframes[year]['per']['YEAR'] = int(year)
    dataframes[year]['fam']['YEAR'] = int(year)

# 2019-2024 uses a different format codex and doesn't need to be split

# 2019-2024 split RTM_VAL into ANN_VAL and DBTN_VAL; additionally added CAP_VAL which we are interested in.
person_cols_split_retirement = ['PERIDNUM', 'PF_SEQ', 'PH_SEQ', 'A_AGE', 'PEAFEVER',
                                'A_HGA', 'A_MJOCC', 'PEARNVAL', 'WSAL_VAL', 'DIV_VAL', 'CAP_VAL', 'ANN_VAL', 'DBTN_VAL']

new_years = ['2019', '2020', '2021', '2022', '2023', '2024']
for year in new_years:
    print(f"Processing {year} data to DataFrame...")
    dataframes[year] = {}
    dataframes[year]['fam'], dataframes[year]['per'] = parse_asec_fixed_width(
        f'output/fwf/{year}_data.DAT', f'output/codex/{year}_famlfmt.txt', f'output/codex/{year}_persfmt.txt', family_cols, person_cols_split_retirement, r"([\w-]+)\s+(\d+)\s+(\d+)")
    
    # set RTM_VAL equal to ANN_VAL + DBTN_VAL
    dataframes[year]['per']['RTM_VAL'] = dataframes[year]['per']['ANN_VAL'] + dataframes[year]['per']['DBTN_VAL']
    dataframes[year]['per'] = dataframes[year]['per'][['PERIDNUM', 'PF_SEQ', 'PH_SEQ', 'A_AGE', 'PEAFEVER', 'A_HGA', 'A_MJOCC', 'PEARNVAL', 'WSAL_VAL', 'CAP_VAL', 'DIV_VAL', 'RTM_VAL']]
    
    # add YEAR to dataframe
    dataframes[year]['per']['YEAR'] = int(year)
    dataframes[year]['fam']['YEAR'] = int(year)

## Merging all datasets

In [63]:
all_fam_dfs = []
all_per_dfs = []

for year in dataframes:
    all_fam_dfs.append(dataframes[year]['fam'])
    all_per_dfs.append(dataframes[year]['per'])

# Concatenate all dataframes by record
merged_fam = pd.concat(all_fam_dfs, ignore_index=True)
merged_per = pd.concat(all_per_dfs, ignore_index=True)

## Add Pew research center income classes

In [240]:
def classify_income_group(df):
    income_classes = []

    for year in df['YEAR'].unique():
        year_data = df[df['YEAR'] == year]
        median_income = year_data['ADJUSTED_INC'].median()

        # Define ranges (based on Pew logic)
        lower = year_data['ADJUSTED_INC'] < 0.67 * median_income
        middle = (year_data['ADJUSTED_INC'] >= 0.67 * median_income) & (year_data['ADJUSTED_INC'] <= 2 * median_income)
        upper = year_data['ADJUSTED_INC'] > 2 * median_income

        income_group = pd.Series(index=year_data.index, dtype="object")
        income_group[lower] = 'Lower'
        income_group[middle] = 'Middle'
        income_group[upper] = 'Upper'

        income_classes.append(income_group)

    df['INCOME_CLASS'] = pd.concat(income_classes).sort_index()
    return df

### Add extra feature aggregates to family record
 ## ---------------------------------------------------------------------------------------------------------------------------------------------
grouped_per = merged_per.groupby(['YEAR', 'PH_SEQ', 'PF_SEQ'])[['CAP_VAL', 'DIV_VAL', 'RTM_VAL']].sum()

fam_plus = pd.merge(merged_fam, grouped_per, left_on=['YEAR', 'FH_SEQ', 'FFPOS'], right_on=['YEAR', 'PH_SEQ', 'PF_SEQ'], how='inner')

fam_plus.rename(columns = {'CAP_VAL': 'CAP_TOT', 'DIV_VAL': 'DIV_TOT', 'RTM_VAL': 'RTM_TOT'}, inplace = True)
fam_plus['ADJUSTED_INC'] = fam_plus['FTOTVAL'] / (fam_plus['FPERSONS'])**.5

fam_plus = classify_income_group(fam_plus)

### Merging family and person dataframes
 ## ---------------------------------------------------------------------------------------------------------------------------------------------
merged_asec = pd.merge(merged_per, fam_plus, left_on=['YEAR', 'PH_SEQ', 'PF_SEQ'], right_on=['YEAR', 'FH_SEQ', 'FFPOS'], how='inner')

first_cols = ['YEAR', 'PH_SEQ', 'PF_SEQ']

other_cols = [col for col in merged_asec.columns if col not in first_cols]

# Reorder
merged_asec = merged_asec[first_cols + other_cols]

## Save merged dataframe to .csv file for efficient loading

In [241]:
fam_plus.to_csv('output/merged/merged_fam.csv', index=False)
merged_asec.to_csv('output/merged/merged_asec.csv', index=False)